# Model Save & Load Practice

In [ ]:
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

## Define VGG11

In [ ]:
class VGG11(nn.Module):
    
    def __init__(self, num_classes=1000):
        super().__init__()
        self.conv1_1 = self.make_conv_relu(3, 64)
        
        self.conv2_1 = self.make_conv_relu(64, 128)

        self.conv3_1 = self.make_conv_relu(128, 256)
        self.conv3_2 = self.make_conv_relu(256, 256)

        self.conv4_1 = self.make_conv_relu(256, 512)
        self.conv4_2 = self.make_conv_relu(512, 512)

        self.conv5_1 = self.make_conv_relu(512, 512)
        self.conv5_2 = self.make_conv_relu(512, 512)
        
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.relu = nn.ReLU()
        
        self.linear_layers = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout2d(0.5),
            nn.Linear(4096, num_classes)
        )

    def make_conv_relu(self, in_channels, out_channel):
        layers = []
        layers += [nn.Conv2d(in_channels, out_channel, kernel_size=3, padding=1),  #using kernel size 3, padding 1 -> keep the spatial dimension of tensor
                   nn.ReLU()]
        return nn.Sequential(*layers)
    
        
    def forward(self, x):
        # x shape: [Batchsize, 3, 32, 32]
        
        x = self.conv1_1(x) # [Batchsize, 64, 32, 32]
        x = self.maxpool(x) # [Batchsize, 64, 16, 16]
        
        x = self.conv2_1(x) # [Batchsize, 128, 16, 16]
        x = self.maxpool(x) # [Batchsize, 128, 8, 8]
        
        x = self.conv3_1(x) # [Batchsize, 256, 8, 8]
        x = self.conv3_2(x) # [Batchsize, 256, 8, 8]
        x = self.maxpool(x) # [Batchsize, 256, 4, 4]
        
        x = self.conv4_1(x) # [Batchsize, 512, 4, 4]
        x = self.conv4_2(x) # [Batchsize, 512, 4, 4]
        x = self.maxpool(x) # [Batchsize, 512, 2, 2]
        
        x = self.conv5_1(x) # [Batchsize, 512, 2, 2]
        x = self.conv5_2(x) # [Batchsize, 512, 2, 2]
        x = self.maxpool(x) # [Batchsize, 512, 1, 1]

        x = x.view(x.size(0), -1) # [Batchsize, 512]
        
        x = self.linear_layers(x) # [Batchsize, num_classes]

        
        return x

In [ ]:
vgg = VGG11(10)
summary(vgg, batch_size=-1, input_size=(3, 32, 32), device='cpu')

## python dict type
- key 와 value로 이루어진 datatype
- torch의 weight는 key : layer 이름 과 value : weights 의 형태로 구성

In [ ]:
# dict_ex, print dict, show state_dict, weights for layer

In [ ]:
# make checkpoints dir, save & load epoch 0 state_dict

In [ ]:
# Loading and normalizing CIFAR-10
import torch
import torchvision
import torchvision.transforms as transforms

import torch.optim as optim


transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #normalization value for cifar10
    ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                         shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg.parameters(), lr=0.001)

print(len(trainloader))
print(len(testloader))

## Save models during training

In [ ]:
### Train the network
vgg.to(device)
vgg.train()
epochs = 3
for epoch in range(1, epochs+1):
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # implement
        # forward + backward + optimize + loss aggregate

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # print statistics
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                 (epoch, i + 1, running_loss / 100))
            running_loss = 0.0
    
    #save models
    
    torch.save(
        {
            'state_dict': vgg.state_dict(),
            'epoch': epoch
        },
        f'./checkpoints/vgg_epoch{epoch}.pth')
    torch.save(vgg.state_dict(), f'./checkpoints/vgg_latest.pth')
    
print('Finished Training')

In [ ]:
#load model
vgg_before = VGG11(10)
checkpoint = torch.load("./checkpoints/vgg_epoch1.pth")

In [ ]:
vgg_before.load_state_dict(checkpoint['state_dict'])

In [ ]:
state_dict_before = vgg_before.state_dict()
state_dict_after = vgg.state_dict()

print(state_dict_before['conv1_1.0.weight'][0,0])
print(state_dict_after['conv1_1.0.weight'][0,0])